In [1]:
# Download Hive 3.1.4
!wget https://downloads.apache.org/hive/hive-3.1.4/apache-hive-3.1.4-bin.tar.gz
!tar -xzf apache-hive-3.1.4-bin.tar.gz


--2025-11-11 14:58:35--  https://downloads.apache.org/hive/hive-3.1.4/apache-hive-3.1.4-bin.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.208.237, 135.181.214.104, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.208.237|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-11-11 14:58:35 ERROR 404: Not Found.

tar (child): apache-hive-3.1.4-bin.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
%%bash
# =============================
# 🧩 STEP 1: Install dependencies
# =============================
apt-get install openjdk-11-jdk-headless -qq > /dev/null

# =============================
# 🧩 STEP 2: Install Hadoop 3.3.6
# =============================
wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
tar -xzf hadoop-3.3.6.tar.gz -C /usr/local/
mv /usr/local/hadoop-3.3.6 /usr/local/hadoop

# =============================
# 🧩 STEP 3: Download Hive 3.1.3 from a verified mirror
# =============================
wget -q https://archive.apache.org/dist/hive/hive-3.1.3/apache-hive-3.1.3-bin.tar.gz

# Verify download (should print file size)
ls -lh apache-hive-3.1.3-bin.tar.gz

# Extract Hive to /usr/local
tar -xzf apache-hive-3.1.3-bin.tar.gz -C /usr/local/
mv /usr/local/apache-hive-3.1.3-bin /usr/local/hive

# Verify Hive folder
ls /usr/local/hive | head


In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/usr/local/hadoop"
os.environ["HIVE_HOME"] = "/usr/local/hive"
os.environ["PATH"] += ":/usr/local/hadoop/bin:/usr/local/hive/bin"

!echo $JAVA_HOME
!echo $HADOOP_HOME
!echo $HIVE_HOME
!ls /usr/local/hive/bin | head -n 10


In [19]:
!tar -tzf apache-hive-3.1.3-bin.tar.gz | head


tar (child): apache-hive-3.1.3-bin.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [18]:
!schematool -dbType derby -initSchema


/bin/bash: line 1: schematool: command not found


In [ ]:
!hive -e """
CREATE DATABASE testdb;
USE testdb;

CREATE TABLE sample (id INT, name STRING);
INSERT INTO TABLE sample VALUES (1,'Arun'),(2,'Bala');
SELECT * FROM sample;
"""


In [9]:
data = """1,Arun,IT,35000
2,Bala,HR,28000
3,Kiran,IT,42000
4,Sara,Finance,39000
5,Devi,HR,31000
6,Ram,Finance,45000
"""

with open("/content/employee.txt", "w") as f:
    f.write(data)

!cat /content/employee.txt


1,Arun,IT,35000
2,Bala,HR,28000
3,Kiran,IT,42000
4,Sara,Finance,39000
5,Devi,HR,31000
6,Ram,Finance,45000


In [10]:
# Initialize Derby metastore
!schematool -dbType derby -initSchema


/bin/bash: line 1: schematool: command not found


In [ ]:
!hive -e """
CREATE DATABASE employee_db;
USE employee_db;

CREATE TABLE employees (
  id INT,
  name STRING,
  dept STRING,
  salary FLOAT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

LOAD DATA LOCAL INPATH '/content/employee.txt' INTO TABLE employees;

SELECT * FROM employees;
"""


In [ ]:
!hive -e """
USE employee_db;

-- Create View
CREATE VIEW high_salary AS
SELECT name, dept, salary FROM employees WHERE salary > 40000;

-- Query View
SELECT * FROM high_salary;

-- Use built-in function (AVG)
SELECT dept, AVG(salary) AS avg_salary FROM employees GROUP BY dept;

-- Create Index (for demo)
CREATE INDEX idx_salary
ON TABLE employees (salary)
AS 'COMPACT' WITH DEFERRED REBUILD;

-- Rebuild index
ALTER INDEX idx_salary ON employees REBUILD;
"""


In [ ]:
!hive -e """
SHOW DATABASES;
USE employee_db;
SHOW TABLES;
DESCRIBE employees;
SHOW INDEXES ON employees;
"""
